# PlotMS in a Notebook

Original Author: drs@nrao.edu



## Description

X11 is a client/server windowing system. The client communicates with the server using the X Window System Core Protocol. This architecture allows the X11 server to accept drawing commands from a remote client and render the GUI drawing commands on the hardware which the server controls/uses, e.g. monitor, keyboard and mouse. The X11 server renders the drawing commands onto a frame buffer. This buffer allows for caching drawing results. At the appropriate time, the buffer contents are flushed video RAM.

This architecture allowed for the implementation of a virtual Frame Buffer and an accompanying virtual server. This allows X11 applications to run without actual display hardware. This is the only way to use X11 (including Qt) applications in the context of a Jupyter notebook. There is no other way.

If someone implemented an X11 server designed for use with Jupyter notebooks, the display of X11 applications within the notebook may be possible. This potential server would behave like XQuartz (the X11 server implementation for MacOS which renders X11 widgets using the native GUI environment on MacOS).

This notebook shows how virtual frame buffers can be used to run X11 applications within a Jupyter notebook. The application used is casaplotms, a Qt application.

The first step is to perform som configuration steps:

In [1]:
# Installation
import os
print("performing setup (tasks a few minutes)...")
os.system('pip3.12 install --upgrade pip wheel')
os.system('apt-get install xvfb')
os.system('pip3.12 install casaconfig==1.4.0')
os.system('pip3.12 install casatools==6.7.2.42')
os.system('pip3.12 install casatasks==6.7.2.42')
os.system('pip3.12 install casaplotms==2.7.4')
os.system('pip3.12 install pyvirtualdisplay')
print('complete')

performing setup (tasks a few minutes)...
complete


In [2]:
print("updating casadata")
from casaconfig import pull_data
# populate some custom location with the data contents
pull_data('./casadata')

!ls ./casadata
print('complete')

updating casadata
pull_data using version casarundata-2025.08.07-1.tar.gz, acquiring the lock ... 
downloading casarundata contents to  ./casadata (333M) ... done
casarundata installed casarundata-2025.08.07-1.tar.gz at ./casadata
alma	  data_update.lock  ephemerides  gui   readme.txt
catalogs  demo		    geodetic	 nrao
complete


In [3]:
print("configuring casa...")
## For google colab
mydatapath = '/content/.casa/data'
## For a local install (choose any writeable path for upto 1GB of metadata)
#mydatapath = '<PATHNAME>/data'

import pathlib
from casaconfig import config
if not pathlib.Path(mydatapath).exists():
  pathlib.Path(mydatapath).mkdir(parents=True)
config.measurespath=mydatapath
print("complete")

configuring casa...
complete


## Setup Virtual Frame Buffer
The pyvirtualdisplay package can be used to configure and launch a virtual frame buffer server. However, the way new processes are directed to the new frame buffer server is via a unix shell environment variable, DISPLAY. This means that this frame buffer server will be used by all X11 processes. This makes it difficult to encapsulate this pattern.

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0,size=(2048,2048))
display.start( )

## Get Data

Somehow data must be made available...

In [5]:
print("download some data...")
os.system("wget https://bulk.cv.nrao.edu/almadata/public/casaguides/FirstLook_TWHya_Band7_6.6.1/twhya_calibrated_unflagged.ms.tar")
os.system("tar -xvf twhya_calibrated_unflagged.ms.tar")
print('complete')

download some data...
complete


## Plot Data

Start X11 application and plot data...

In [24]:
from casaplotms import plotms
print('making the plot...')

plotms(vis='twhya_calibrated_unflagged.ms',plotfile='twhya_calibrated_unflagged.jpg',showgui=False,highres=True,width=600,height=350,overwrite=True)
print('complete')

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Display the Exported Raster Image

In [ ]:
from IPython.display import Image
Image(filename="sis14_twhya_calibrated_flagged.jpg")